<a href="https://colab.research.google.com/github/Ghonem22/Speed-up-your-dataframe-analysis-till-1500X/blob/main/Speed_up_your_dataframe_analysis_till_1500X_using_swifter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## install required library


In [1]:
!pip install swifter

     |████████████████████████████████| 647 kB 5.3 MB/s 
     |████████████████████████████████| 280 kB 44.7 MB/s 
     |████████████████████████████████| 136 kB 40.0 MB/s 
  Created wheel for swifter: filename=swifter-1.1.3-py3-none-any.whl size=13078 sha256=d6637cb89aa872a103801ffb517a462222ad0b247ff2285ab4119605f06cccdd
  Stored in directory: /root/.cache/pip/wheels/e8/1a/1b/46aa7b0fac16d3ce1bccaac14bfe4fad972642217ae78d8160
Successfully built swifter
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


## Required libraries

In [2]:
import shutil
import swifter
import time

## Download and unzip data

In [3]:
!wget https://www.stats.govt.nz/assets/Uploads/New-Zealand-business-demography-statistics/New-Zealand-business-demography-statistics-At-February-2021/Download-data/Geographic-units-by-industry-and-statistical-area-2000-2021-descending-order-CSV.zip

--2022-04-15 14:27:49--  https://www.stats.govt.nz/assets/Uploads/New-Zealand-business-demography-statistics/New-Zealand-business-demography-statistics-At-February-2021/Download-data/Geographic-units-by-industry-and-statistical-area-2000-2021-descending-order-CSV.zip
Resolving www.stats.govt.nz (www.stats.govt.nz)... 45.60.11.104
Connecting to www.stats.govt.nz (www.stats.govt.nz)|45.60.11.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22898963 (22M) [application/zip]
Saving to: ‘Geographic-units-by-industry-and-statistical-area-2000-2021-descending-order-CSV.zip’

Geographic-units-by 100%[===================>]  21.84M  7.83MB/s    in 2.8s    

2022-04-15 14:27:54 (7.83 MB/s) - ‘Geographic-units-by-industry-and-statistical-area-2000-2021-descending-order-CSV.zip’ saved [22898963/22898963]



In [4]:
shutil.unpack_archive("/content/Geographic-units-by-industry-and-statistical-area-2000-2021-descending-order-CSV.zip", "/content")

In [5]:
import pandas as pd
df = pd.read_csv("/content/Data7602DescendingYearOrder.csv")
print(f"this dataframe has {len(df)} rows \n\n\n")
df.head()

this dataframe has 5704247 rows 





,anzsic06,Area,year,geo_count,ec_count
0,A,A100100,2021,90,200
1,A,A100200,2021,141,210
2,A,A100300,2021,6,25
3,A,A100400,2021,54,65
4,A,A100500,2021,63,95


# Use simple summation to Compare the diffrence in time between itration, lmbda, swifter, and Vectorixation

## Itrating over the dataframe using iterrows()

In [6]:
def sum(x, y):
    return x * 15 + y ** 2

In [15]:
# initalize a new column "new"
df['new'] = -1

t = time.time()
for index, row in df.iterrows():

    df.loc[index, 'new'] = sum(df.loc[0, 'geo_count'], df.loc[0, 'ec_count'])

    # print the time every 1000 rows
    if index % 1000 == 0 and index != 0:
        print(f"the code take {time.time() - t} to process {index} rows ")

print(f"the complete data is processed after {time.time() - t} seconds")

the code take 47.36810922622681 to process 1000 rows 
the code take 89.51108074188232 to process 2000 rows 


KeyboardInterrupt: ignored

**As we see, processing just 2000 rows take about 89 seconds, So it's ver unpractical to use that in processing big data**

# lambda

In [16]:
del df['new']

In [17]:
t = time.time()
df['new'] = df.apply(lambda x: sum(x['geo_count'], x['ec_count']), axis=1)
time.time() - t

80.58791661262512

In [18]:
t = time.time()
df['new'] = df.apply(lambda x: sum(x.geo_count, x.ec_count) , axis=1)
time.time() - t

121.99550247192383

**We can see that calling columns as "x['geo_count']" is a bit faster compared to calling them as "x.geo_count"**

## lambda with **swifter**

In [24]:
del df['new']

In [25]:
t = time.time()
df['new'] = df.swifter.apply(lambda x: sum(x['geo_count'], x['ec_count']), axis=1)
time.time() - t

0.06731629371643066

In [26]:
121.99550247192383 / 0.06731629371643066

1812.2730135118384

**As We see, Just by adding swifter, the code is faster between 900X to 1398X**

### Why is **swifter** that faster?

**The answer is simple, swifter support Parallelization, so the processing become faster when you deal with big data**

## Can we make the code more faster?



**The answer in vectorization:**
* You should try first to vectorize your process, this is the optimal solution
* in some cases, it will be hard to do that, so you can use swifter to speed up your code using the advantage of Parallelization 

In [ ]:
t = time.time()
df['new2'] = df['geo_count'] * 15 + df['ec_count'] ** 2
time.time() - t

0.07243514060974121

In [ ]:
df['new'] == df['new2']

0          True
1          True
2          True
3          True
4          True
           ... 
5704242    True
5704243    True
5704244    True
5704245    True
5704246    True
Length: 5704247, dtype: bool

## resources:

1. https://towardsdatascience.com/speed-up-your-pandas-processing-with-swifter-6aa314600a13

2. https://towardsdatascience.com/do-you-use-apply-in-pandas-there-is-a-600x-faster-way-d2497facfa66